In [1]:
# https://www.youtube.com/watch?v=YrMy-BAqk8k&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=15
# https://www.tensorflow.org/datasets/catalog/overview

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=["train", "test"],# може бути validatioin в деяких
    shuffle_files=True,
    as_supervised=True,# to return (img, labe) otherwise will return dictionary
    with_info=True,
)


# fig = tfds.show_examples(ds_train,ds_info,rows=4, cols=4) #показатил 4 4 кількість рядків і колонок для прикладів
# print(ds_info)



In [2]:
def normalize_img(image, label):
    #normalize images
    return tf.cast(image,tf.float32)/255.0, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)# другий параметр щоб паралельно запустити
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)

model = keras.Sequential([
    keras.Input((28,28,1)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(10,activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

model.fit(ds_train, epochs=5, verbose=1) # ds_train того що в нас там вже є turple

model.evaluate(ds_test)



Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8721 - loss: 0.4646
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9705 - loss: 0.1074
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9806 - loss: 0.0719
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9851 - loss: 0.0516
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9877 - loss: 0.0422
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9798 - loss: 0.0582


[0.060848284512758255, 0.9793000221252441]

In [ ]:
import tensorflow_datasets as tfds

# навчання на емоційну окраску тексту
# !!! недороблене
(ds_train, ds_test), ds_info = tfds.load(
    "imdb_reviews",
    split=["train","test"],
    shuffle_files = True,
    as_supervised=True,
    with_info=True
)

# print(ds_info)

# for text, label in ds_train:
#     print(text)
#     import sys
#     sys.exit
    
tokenizer = tfds.features.text.Tokenizer()

def build_vocabulary():
    vocabulary = set()
    for text, _ in ds_train:
        vocabulary.update(tokenizer.tokenize(text.numpy().lower()))
    return vocabulary

vocabulary = build_vocabulary()

encoder = tfds.features.text.TokenTextEncoder(
    list(vocabulary), oov_token="<UNK>", lowercase=True, tokenizer=tokenizer
)

def my_encoding(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

def encode_map_fn(text,label):
    encoded_text, label = tf.py_function(
        my_encoding, inp = [text, label], Tout=(tf.int64, tf.int64)
    )
    
    encoded_text.set_shape([None])
    label.set_shape([])
    
    return encoded_text, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(encode_map_fn, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(10000)
ds_train = ds_train.padded_batch(32,padded_shapes=([None],()))
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(encode_map_fn)
ds_test = ds_test.padded_paatch(32, padded_shapes([None],()))

model = keras.Sequential([
    layers.Masking(mask_value=0),
    layers.embedding(input_dim=len(vocabulary) + 2,output_dim=32),
    #BATCH_SIZE * 1000 -> BATCH_SIZE *1000 *32
    layers.GlobalAvaragePooling1D(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1),
])

model.compile(
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(3e-4, clipnorm=1),
    metrics=["accuracy"]
)

model.fit(ds_train,epochs=10,verbose=1)
model.evaluate(ds_test)


NameError: name 'texts' is not defined